# Resolvendo Pacman
_The definition of the problem, the solution, and the results obtained must be presented in a report created as a
Jupyter notebook. Please, make sure you put the graphs, tables, comparisons, and critical analysis in the notebook.
The report should clearly indicate what the contribution of each team member was._

O objetivo deste trabalho é de resolver o jogo Pacman, de forma estática, utilizando os algoritmos de busca aprendidos nas aulas de MC906/MO416.

## Algoritmos utilizados
1. Breadth first search
2. Depth first search
3. Best First search
4. A*
5. Hill climbing search (ou simulated annealing, ainda não sei)
# **explicar a escolha dos métodos depois**

## Heurísticas

As Heurísticas elaboradas foram determinantes, principalmente para os Informed Search Algorithms, onde seu resultado h(n) é contabilizado juntamente do path cost   g(n) para o agente decidir o próximo passo a ser avaliado. Podendo ou não indicar a melhor solução.

Para o problema do Pacman nossas heurísticas estão voltadas para achar o menor caminho possível para nosso agente chegar ao *goal*, não foram parametrizados as moedas espalhadas pelo mapa.

Com uma busca pela bibliografia e na internet achamos duas heurísticas bastante utilizadas em problemas desse tipo.

### Distância Manhattan

Baseado na "real" distância que o agente irá percorrer. Considera o número de passos que serão dados, o número de ações que serão tomadas.

O processamento é um pouco mais custoso do que o normal, devido a essa metodologia.



In [ ]:
   # Calcula a distancia entre duas posicoes, valido apenas para posicoes que fazem parte da sol
    @functools.lru_cache(maxsize=4096)
    def manhattan_distance(self, position1, position2, lim=100000):
        if position1 == position2:
            return 0
   # dicicionario com formato { item: [distancia, visitado (0 = nao, 1 = sim)], ... }
        d = dict.fromkeys(self.reachable_positions(position1), [math.inf, 0])
        d[position1] = [0, 0]

        i = 0
        while min({l[1] for l in d.values()}) == 0 and i < lim:
            m = math.inf
            current = None
            for k, v in d.items():
                if v[1] == 0 and v[0] < m:
                    current = k
                    m = v[0]

            d[current] = [m, 1]

            neighbors = self.adjacent(current)
            for neighbor in neighbors:
                if d[neighbor][1] == 0 and m + 1 < d[neighbor][0]:
                    d[neighbor] = [m + 1, 0]
                    if neighbor == position2:
                        return m + 1

            current = None
            i = i + 1

        try:
            return d[position2][0]
        except KeyError:
            return math.inf``

O algoritmo acima checa a **distância de manhattan** entre dois pontos.
Inicialmente, ele checa se os pontos passados se referem a mesma posição, em caso positivo, já temos nossa resposta! (e nos poupa um bom processamento). Em caso negativo manhattan_distance usa uma função auxiliar (*reachable_positions*) que nos retorna um dicionário onde dada a chave (coordenadas da respectiva posição) temos um valor (uma lista (distancia, visitado)).
Enquanto não existir elementos ainda não visitados, avaliamos o **mais próximo** dentre estes,  da posição corrente da nossa busca.

Com esse nó avaliado no loop, para cada posição adjacente não avaliada, incrementa-se a distância respectiva.

Esse processo continua até que se visite a posição dada por _position2_ 

Ou seja, em resumo, simulamos os passos a serem dados para nosso agente chegar de um ponto A (*position1*) a um ponto B (*position2*) e projetamos o menor caminho possível, sustentado pela estratégia de que iniciamos cada uma das "buscas parciais" do nó com a menor distância, nos termos de manhattan, a cada iteração.

### Distância Euclidiana


In [ ]:
      def euclidean_distance(self, position1, position2):
        if self.reachable(position1, position2) is False:
            return math.inf
        
        deltaX1 = (position2[0] - position1[0])**2
        deltaY1 = (position2[1] - position1[1])**2
        return (deltaX1 + deltaY1)**0.5

Essa heurística é bem mais direta em termos de lógica e processamento de dados. 

A distância euclidiana avalia simplismente, a distância em linha reta, do ponto A ao ponto B, que no caso de nosso algoritmo de busca seriam o ponto de avaliação atual ao goal.

Em primeiro lugar, checamos se os dois pontos são alcançáveis. Se sim, calculamos a distância segundo a fórmula de Euclides:

$\sqrt{(x2 - x1)² + (y2 - y1)²)}$


## Representação do problema e seus estados

Como forma de facilitar nosso trabalho, usamos a estrutura da ferramenta AIMA, dada como auxílio para o desenvolvimento do nosso projeto. Aproveitamos a classe **__Node__** para armazenar o estado, que nada mais seria que as coordenadas (x, y) descrevendo a posição no maze.

Essa classe contém alguns métodos que nos serão muito úteis, como o método *path*, que nos dá o caminho percorrido até ali pela busca, ou o *expand* que nos retorna uma lista de nós que podemos seguir a partir das ações disponíveis.

Quanto aos métodos de busca, por estarmos aproveitando a estrutura dada também seguiu a mesma lógica do AIMA. Algumas alterações foram necessárias...

### TODO

## Goal Test

Simplesmente é passado o path contendo o novo node a ser avaliado e, como estrutura do problema, nosso goal.
Caso o goal esteja contido no path achamos nosso objetivo! O agente então poderá agir.

Aqui é importante lembrar que nosso agente realiza somente buscas offline, ou seja, antes de realizar uma ação, a busca é executada e uma solução é encontrada.

### 1. Breadth first search

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.01 segundos, iterações: 136, posições distintas visitadas: 14

*Problema2:* Tempo de execução: 0.01 segundos, iterações: 159, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.003 segundos, iterações: 66, posições distintas visitadas: 14

### 2. Depth first search

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.009 segundos, iterações: 317, posições distintas visitadas: 24

*Problema2:* Tempo de execução: 0.01 segundos, iterações: 348, posições distintas visitadas: 27

*Problema3:* Tempo de execução: 0.002 segundos, iterações: 94, posições distintas visitadas: 18

### 3. Greedy Best first search

O _Greedy Best First Search Algoritm_ funciona de maneira muito parecida com o _A* Search_. A partir de um nó inicial, avalia-se o custo f(n) do caminho a cada um de seus possíveis destinos, de acordo com as ações que poderão ser tomadas.

Onde,

  f(n) = g(n) + h(n) 
 
  g(n) é o custo do caminho tomado e h(n) é o custo segundo a heurística adotada para a avaliação.


Seu nome greedy (ganancioso) se dá pelo fato de que após uma ação tomada, não faz mais parte da avaliação (até que se comprove que o caminho não se dá até o goal) os custos perantes as demais ações possíveis, os outros nós adjacentes no nosso caso.

<p align="center">
  <img width="400" height="400" src="Greedy.png">
</p>

Acima vemos uma imagem que ilustra o comportamento do algoritmo, independente de que outros nós possam formar um caminho mais caro, por agora, mas mais vantajoso como solução completa.

**Completo:** Não.

**Ótimo:** Não.

**Complexidade de tempo:** O(b^m). 

**Complexidade de memória:** O(b^m).

Em ambos b é a quantidade de nós que a atual posição pode expandir e m refere-se à profundidade da busca.
Mas é importante ressaltar que em alguns casos, a situação pode favorecer e se tornar um algoritmo bem eficiente, como vimos através dos resultados obtidos.
O grande problema da greedy solution é o fato de não ter controle em situações de espaço infinito, ou em caminhos sem solução. Outra situação ruim é que nem sempre optará pelo caminho menos custoso, o que no nosso caso não chegou a ser um problema, mas em situações de maior escala podem vir a ser um grande problema.

**Resultados:**

*Problema1:* Tempo de execução: 0.003 segundos, iterações: 47, posições distintas visitadas: 14

*Problema2:* Tempo de execução: 0.004 segundos, iterações: 77, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.001 segundos, iterações: 47, posições distintas visitadas: 14

### 4. A*

**Completo:** Sim.

**Ótimo:** Não.

**Complexidade de tempo:** Toda vez que é feita uma expansão até 3 novos nós são adicionados à fila de prioridade, portanto **O**($3^{d+1}$), onde d+1 é a profundidade da solução mais rasa (mais próxima do estado inicial).

**Complexidade de memória:** O algoritmo armazena os nós expandidos em uma fila de prioridade e os visitados em uma lista, poranto **O**($3^{d+1}$), igual à complexidade de tempo.

**Resultados:**

*Problema1:* Tempo de execução: 0.13 segundos, iterações: 156, posições distintas visitadas: 18

*Problema2:* Tempo de execução: 0.21 segundos, iterações: 182, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.02 segundos, iterações: 47, posições distintas visitadas: 14

### 5. Hill climbing search

**Completo:** TODO.

**Ótimo:** TODO.

**Complexidade de tempo:** TODO.

**Complexidade de memória:** TODO.

**Resultados:**

*Problema1:* Tempo de execução: 0.004 segundos, iterações: 173, posições distintas visitadas: 18

*Problema2:* Tempo de execução: 0.007 segundos, iterações: 182, posições distintas visitadas: 23

*Problema3:* Tempo de execução: 0.003 segundos, iterações: 71, posições distintas visitadas: 14